Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
train_overfitting_dataset, train_overfitting_labels = train_dataset[:1000], train_labels[:1000]
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape
print 'overfitting set: ', train_overfitting_dataset.shape, train_overfitting_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)
overfitting set:  (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# SGD Logistic model with L2 regularization

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 45.4204
Minibatch accuracy: 14.8%
Validation accuracy: 17.7%
Minibatch loss at step 500 : 0.915054
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 1000 : 0.857227
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 1500 : 0.784715
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 2000 : 0.728269
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 2500 : 0.847331
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 3000 : 0.857854
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Test accuracy: 87.2%


Increased accuracy by about 2%

In [25]:
# Now add regularization to the net


batch_size = 128
num_nodes = 1024
beta1 = 0.0001
beta2 = 0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases1 = tf.Variable(tf.zeros([num_nodes]))
    
  weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights1) + biases1
  
  non_linear = tf.nn.relu(logits)
  
  final_layer = tf.matmul(non_linear, weights2) + biases2
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(final_layer, tf_train_labels)) + beta1 * tf.nn.l2_loss(weights1) \
    + beta2 * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(final_layer)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
    tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 370.529
Minibatch accuracy: 14.1%
Validation accuracy: 22.2%
Minibatch loss at step 500 : 47.4568
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Minibatch loss at step 1000 : 37.102
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 1500 : 35.9374
Minibatch accuracy: 78.1%
Validation accuracy: 79.8%
Minibatch loss at step 2000 : 27.2416
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 2500 : 27.0983
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 3000 : 24.2082
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Test accuracy: 89.4%


Also increased by ~2%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Overfitting, performance on the test and validation sets decreases, minibatch accuracy increased to ~100%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
# Try to introduce dropout on the Net

batch_size = 128
num_nodes = 8192
beta1 = 0.00001
beta2 = 0.00001
beta3 = 0.00001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases1 = tf.Variable(tf.zeros([num_nodes]))
    
  weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  
  hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), 0.5)

  second_layer = tf.matmul(hidden_layer, weights2) + biases2


  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(second_layer, tf_train_labels))

  regular = beta1 * tf.nn.l2_loss(weights1) + beta2 * tf.nn.l2_loss(weights2)
  
  loss += regular

#   Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

  # Optimizer.
#   global_step = tf.Variable(0)  # count the number of steps taken.
#   learning_rate = tf.train.exponential_decay(0.0004, global_step,1000,0.90,staircase=True)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(second_layer)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights1)\
                                                                      + biases1)), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu((tf.matmul(\
                                                                tf_test_dataset, weights1)\
                                                                    + biases1)), weights2) + biases2)

# test_prediction = tf.nn.softmax(
#     tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)


In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1526.32
Minibatch accuracy: 13.3%
Validation accuracy: 14.6%
Minibatch loss at step 500 : 304.205
Minibatch accuracy: 68.8%
Validation accuracy: 81.3%
Minibatch loss at step 1000 : 292.313
Minibatch accuracy: 71.1%
Validation accuracy: 82.7%
Minibatch loss at step 1500 : 212.938
Minibatch accuracy: 78.1%
Validation accuracy: 83.3%
Minibatch loss at step 2000 : 211.478
Minibatch accuracy: 79.7%
Validation accuracy: 84.0%
Minibatch loss at step 2500 : 258.846
Minibatch accuracy: 70.3%
Validation accuracy: 84.3%
Minibatch loss at step 3000 : 270.633
Minibatch accuracy: 73.4%
Validation accuracy: 84.6%
Minibatch loss at step 3500 : 216.991
Minibatch accuracy: 77.3%
Validation accuracy: 84.8%
Minibatch loss at step 4000 : 150.906
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
Minibatch loss at step 4500 : 154.017
Minibatch accuracy: 82.0%
Validation accuracy: 85.1%
Minibatch loss at step 5000 : 159.535
Minibatch accuracy: 80.5%
Validation accurac

Sweet, 91.6 with dropout (1024 nodes).  What happens when number of nodes is increased?  1024-2048, also decreased betas and learning rate, but LR is still constant.  Sweet, 92%, what happens when rad LR is introduced?  Ooh, 83.9, no good!  92.6% after increasing all the way to 4096!   Now change back to 1024 hidden nodes, and switch from ReLU to tanh: Wow, no good, 66.9%.  Weird, how about ReLU6?  Certainly better at 88.9%.  One more time: Sigmoid, 83.7.  Bizarre.  Okay, back to relu, bumped the hidden layer up to 8192, decreased LR and betas, got 93%, highest yet.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
# Try to add multiple layers


batch_size = 128
num_nodes_1 = 1024
num_nodes_2 = 1024
beta1 = 0.000000001
beta2 = 0.000000001
beta3 = 0.000000001
beta4 = 0.000000001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes_1]))
  biases1 = tf.Variable(tf.zeros([num_nodes_1]))
    
  weights2 = tf.Variable(tf.truncated_normal([num_nodes_1, num_nodes_2]))
  biases2 = tf.Variable(tf.zeros([num_nodes_2]))
    
  weights3 = tf.Variable(tf.truncated_normal([num_nodes_2, num_nodes_2]))
  biases3 = tf.Variable(tf.zeros([num_nodes_2]))
    
  weights4 = tf.Variable(tf.truncated_normal([num_nodes_2, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights2) + biases2)
  
  hidden_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_2, weights3) + biases3), 0.5)
    
  final_layer = tf.matmul(hidden_3, weights4) + biases4

  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(final_layer, tf_train_labels))

  regular = beta1 * tf.nn.l2_loss(weights1) + beta2 * tf.nn.l2_loss(weights2) + beta3 * tf.nn.l2_loss(weights3) \
                        + beta4 * tf.nn.l2_loss(weights4)
  
  loss += regular

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0000004, global_step,1000,0.90,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    


# optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(final_layer)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) \
                                                                                                  + biases1), weights2)\
                                                          + biases2), weights3) + biases3), weights4) + biases4)
    
    
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)\
                                                                                    + biases1), weights2)\
                                                          + biases2), weights3) + biases3), weights4) + biases4)

In [6]:
%%time

num_steps = 15001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 183895.0
Minibatch accuracy: 12.5%
Validation accuracy: 11.2%
Minibatch loss at step 500 : 95406.8
Minibatch accuracy: 18.8%
Validation accuracy: 33.6%
Minibatch loss at step 1000 : 89221.0
Minibatch accuracy: 23.4%
Validation accuracy: 52.3%
Minibatch loss at step 1500 : 72524.9
Minibatch accuracy: 36.7%
Validation accuracy: 60.3%
Minibatch loss at step 2000 : 72578.0
Minibatch accuracy: 39.1%
Validation accuracy: 65.1%
Minibatch loss at step 2500 : 70891.9
Minibatch accuracy: 36.7%
Validation accuracy: 67.7%
Minibatch loss at step 3000 : 53018.3
Minibatch accuracy: 41.4%
Validation accuracy: 69.6%
Minibatch loss at step 3500 : 48522.5
Minibatch accuracy: 50.0%
Validation accuracy: 70.8%
Minibatch loss at step 4000 : 44245.6
Minibatch accuracy: 57.8%
Validation accuracy: 71.7%
Minibatch loss at step 4500 : 38812.9
Minibatch accuracy: 60.2%
Validation accuracy: 72.3%
Minibatch loss at step 5000 : 32246.5
Minibatch accuracy: 62.5%
Validation accura

Why is my performance so much worse with additional layers?  hypothesis: not enough steps for constant learning rate.  Try exponential decay.

Okay, increased the steps, added exp learning rate, increased beta1 and beta2, while leaving beta3 pretty small.  Quite a big improvement.  Dropout still present on the second hidden layer.  Not sure multiple layers of dropout makes any sense.  Got 90.5% with one layer of dropout.  Without any dropout I get 90.8%, 6 minutes of calculations.  Increase to 91.3% (8 mins) with equally sized hidden layers (1024).

I'd like to try a third hidden layer.  I wonder what that would do?  Well this makes no sense.  I added a third hidden layer and the performance plummeted.  Very odd.  Ah-ha!  Possibly my beta coefficients were way too large.  Nope, still too big.  Reduced betas by a large amount and also started the learning rate 100 times lower.  Better results now, no shock, almost anything is better than ~12%.  Got back up to 88.1%, still not quite as good.  Could probably continue to improve via parameter search.